# TimeEmbedding class
* ./common/time_layers.py

embedding층 역할: 단어 입력 -> 면 번째 등장?: 단어 id <br>
embedding 층에 Weight matrix 가 있으면 <br>
Weight Matrix에서 단어에 해당하는 행 뽑아냄. <br>
그 뽑아낸 행이 단어의 벡터표현. 그걸 RNN층으로 보내라 <br>

In [ ]:
class TimeEmbedding:
    def __init__(self, W): # embedding층 입력: weight
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.layers = None
        self.W = W

    def forward(self, xs): # 순전파, 입력: 밑에서 올라오는 단어들
        N, T = xs.shape # batch_size, time_block의 길이
        V, D = self.W.shape # (weight mat안에는 단어의 헥터표현이 각 행에 저장됨); 행개수 = 단어개수, weight mat의 열 크기 = 벡터의 크기(단어의벡터표현)

        out = np.empty((N, T, D), dtype='f') # 3차원 tensor # 단어 id에 해당하는 ....를 채워 넣는...
        # 전체적인 shape은 batch_size * Time_size
        self.layers = [] # 공리스트에 임베딩 인스턴스 차례대로 채워넣을 것

        for t in range(T): # timeblock 길이 (timesize)
            layer = Embedding(self.W) # 임베딩인스턴스를 레이어로 두고
            out[:, t, :] = layer.forward(xs[:, t]) # 순전파, 열자리에 layer넣고 순전파
            self.layers.append(layer) # 방금 만ㄴ든 인스턴스 추가

        return out

    def backward(self, dout): # 역전파, 위에서 흘러들어온 미분 필요
        N, T, D = dout.shape # out의 shape과 동일

        grad = 0
        for t in range(T):
            layer = self.layers[t]
            layer.backward(dout[:, t, :])
            grad += layer.grads[0]

        self.grads[0][...] = grad
        return None # 밑으로 흘러보낼 필요가 없기 때문에 None

# TimeAffine class
* ./common/time_layers.py

In [ ]:
class TimeAffine:
    def __init__(self, W, b):
        self.params = [W, b]
        self.grads = [np.zeros_like(W), np.zeros_like(b)]
        self.x = None

    def forward(self, x): # 순전파 하려면 밑에서 흘러들어온 데이터 필요 (밑: time rnn 층, 거기서 나오는게 hidden_state, 이건 위로도 보내고, 동시에 순환을 시킴)
        N, T, D = x.shape
        W, b = self.params

        rx = x.reshape(N*T, -1) # 각각에 대한 affine변환을 하지 않고 한 번에 하기 위한 행렬 변환
        out = np.dot(rx, W) + b # 한 번에 오른쪽에 Weight mat 곱하고 편향벡터 더하기 (out: affine 변환 결과)
            # affine층에서 나가는건 vocab 사이즈
        self.x = x
        return out.reshape(N, T, -1) # Batch_size * Time_size * vocab_size, 이제 이걸 위 층으로 보냄.

    def backward(self, dout): # 역전파, 위에서 흘러들어온 미분 필요. (위: time Softmax 층)
        x = self.x
        N, T, D = x.shape # shape은 위(time Softmax층)로 보낼 때랑 동일.
        W, b = self.params

        dout = dout.reshape(N*T, -1) # reshape해서 3차원 텐서를 행렬로 만듦. (세로: Batch_size * Time_size, 가로: vocab_size)
            # softmax층에서 내려온 거
        rx = x.reshape(N*T, -1) # 밑에서 time affine 층으로 올라온 거 (밑: time rnn),
            # 3차원 텐서를 행렬로 바꿔줘야 Affine계산을 생각할 수 있음.

        db = np.sum(dout, axis=0)
        dW = np.dot(rx.T, dout) # Weight matrix에대한 미분. 흘러 들어온 미분 dout에 x를 Transpose해서 엇갈려서 (왼쪽에다가) 곱해줌. (3차원 텐서를 행렬로 바꿔준 rx) 
        dx = np.dot(dout, W.T) # x에 대한 미분. 흘러들어온 미분 dout에 W를 Transpose해서 오른쪽에다가 곱해줌.
        dx = dx.reshape(*x.shape) # 행렬을 다시 원래 형태인 3차원텐서형태로 바꿔줌. Batch_size * Time_size * 그리고 안에는 hidden_size
            # hidden_size인 이유: 왜냐면 x로 미분을 해주는 것인데, x: Time Affine 층으로 들어오는 hidden_state들의 모임이기 때문. (하나하나가 hidden_state shape과 동일해야 함.)

        self.grads[0][...] = dW
        self.grads[1][...] = db
        # 편향벡터, Weight mat 구한 것을 self.grad에 덮어써서 이걸 통해서 update

        return dx # 밑에서 올라온 hidden_state에 대한 미분은 리턴해서 밑(time rnn층)으로 흘려보냄.

# TimeSoftmaxWithLoss class
* ./common/time_layers.py

In [ ]:
class TimeSoftmaxWithLoss:
    def __init__(self):
        self.params, self.grads = [], []
        self.cache = None
        self.ignore_label = -1

    def forward(self, xs, ts):
        N, T, V = xs.shape

        if ts.ndim == 3:  # 정답 레이블이 원핫 벡터인 경우
            ts = ts.argmax(axis=2)

        mask = (ts != self.ignore_label)

        # 배치용과 시계열용을 정리(reshape)
        xs = xs.reshape(N * T, V)
        ts = ts.reshape(N * T)
        mask = mask.reshape(N * T)

        ys = softmax(xs)
        ls = np.log(ys[np.arange(N * T), ts])
        ls *= mask  # ignore_label에 해당하는 데이터는 손실을 0으로 설정
        loss = -np.sum(ls)
        loss /= mask.sum()

        self.cache = (ts, ys, mask, (N, T, V))
        return loss

    def backward(self, dout=1):
        ts, ys, mask, (N, T, V) = self.cache

        dx = ys
        dx[np.arange(N * T), ts] -= 1
        dx *= dout
        dx /= mask.sum()
        dx *= mask[:, np.newaxis]  # ignore_labelㅇㅔ 해당하는 데이터는 기울기를 0으로 설정

        dx = dx.reshape((N, T, V))

        return dx

# ./ch05/simple_rnnlm.py
RNNLM: RNN Language Model

In [ ]:
# coding: utf-8
import sys
sys.path.append('..')
import numpy as np
from common.time_layers import *


class SimpleRnnlm:
    def __init__(self, vocab_size, wordvec_size, hidden_size):
        V, D, H = vocab_size, wordvec_size, hidden_size
        rn = np.random.randn

        # 가중치 초기화
        embed_W = (rn(V, D) / 100).astype('f')
        rnn_Wx = (rn(D, H) / np.sqrt(D)).astype('f')
        rnn_Wh = (rn(H, H) / np.sqrt(H)).astype('f')
        rnn_b = np.zeros(H).astype('f')
        affine_W = (rn(H, V) / np.sqrt(H)).astype('f')
        affine_b = np.zeros(V).astype('f')

        # 계층 생성
        self.layers = [
            TimeEmbedding(embed_W),
            TimeRNN(rnn_Wx, rnn_Wh, rnn_b, stateful=True),
            TimeAffine(affine_W, affine_b)
        ]
        self.loss_layer = TimeSoftmaxWithLoss()
        self.rnn_layer = self.layers[1]

        # 모든 가중치와 기울기를 리스트에 모은다.
        self.params, self.grads = [], []
        for layer in self.layers:
            self.params += layer.params
            self.grads += layer.grads

    def forward(self, xs, ts):
        for layer in self.layers:
            xs = layer.forward(xs)
        loss = self.loss_layer.forward(xs, ts)
        return loss

    def backward(self, dout=1):
        dout = self.loss_layer.backward(dout)
        for layer in reversed(self.layers):
            dout = layer.backward(dout)
        return dout

    def reset_state(self):
        self.rnn_layer.reset_state()

# ./ch05/train_custom_loop.py

In [ ]:
# coding: utf-8
import sys
sys.path.append('..')
import matplotlib.pyplot as plt
import numpy as np
from common.optimizer import SGD
from dataset import ptb
from simple_rnnlm import SimpleRnnlm


# 하이퍼파라미터 설정
batch_size = 10
wordvec_size = 100
hidden_size = 100 # RNN의 은닉 상태 벡터의 원소 수
time_size = 5     # Truncated BPTT가 한 번에 펼치는 시간 크기
lr = 0.1
max_epoch = 100

# 학습 데이터 읽기(전체 중 1000개만)
corpus, word_to_id, id_to_word = ptb.load_data('train')
corpus_size = 1000
corpus = corpus[:corpus_size]
vocab_size = int(max(corpus) + 1)

xs = corpus[:-1]  # 입력
ts = corpus[1:]   # 출력(정답 레이블)
data_size = len(xs)
print('말뭉치 크기: %d, 어휘 수: %d' % (corpus_size, vocab_size))

# 학습 시 사용하는 변수
max_iters = data_size // (batch_size * time_size)
time_idx = 0
total_loss = 0
loss_count = 0
ppl_list = []

# 모델 생성
model = SimpleRnnlm(vocab_size, wordvec_size, hidden_size)
optimizer = SGD(lr)

# 미니배치의 각 샘플의 읽기 시작 위치를 계산
jump = (corpus_size - 1) // batch_size
offsets = [i * jump for i in range(batch_size)]

for epoch in range(max_epoch):
    for iter in range(max_iters):
        # 미니배치 취득
        batch_x = np.empty((batch_size, time_size), dtype='i')
        batch_t = np.empty((batch_size, time_size), dtype='i')
        for t in range(time_size):
            for i, offset in enumerate(offsets):
                batch_x[i, t] = xs[(offset + time_idx) % data_size]
                batch_t[i, t] = ts[(offset + time_idx) % data_size]
            time_idx += 1

        # 기울기를 구하여 매개변수 갱신
        loss = model.forward(batch_x, batch_t)
        model.backward()
        optimizer.update(model.params, model.grads)
        total_loss += loss
        loss_count += 1

    # 에폭마다 퍼플렉서티 평가
    ppl = np.exp(total_loss / loss_count)
    print('| 에폭 %d | 퍼플렉서티 %.2f'
          % (epoch+1, ppl))
    ppl_list.append(float(ppl))
    total_loss, loss_count = 0, 0

# 그래프 그리기
x = np.arange(len(ppl_list))
plt.plot(x, ppl_list, label='train')
plt.xlabel('epochs')
plt.ylabel('perplexity')
plt.show()

# ./ch05/train.py

In [ ]:
# coding: utf-8
import sys
sys.path.append('..')
from common.optimizer import SGD
from common.trainer import RnnlmTrainer
from dataset import ptb
from simple_rnnlm import SimpleRnnlm


# 하이퍼파라미터 설정
batch_size = 10
wordvec_size = 100
hidden_size = 100  # RNN의 은닉 상태 벡터의 원소 수
time_size = 5  # RNN을 펼치는 크기
lr = 0.1
max_epoch = 100

# 학습 데이터 읽기
corpus, word_to_id, id_to_word = ptb.load_data('train')
corpus_size = 1000  # 테스트 데이터셋을 작게 설정
corpus = corpus[:corpus_size]
vocab_size = int(max(corpus) + 1)
xs = corpus[:-1]  # 입력
ts = corpus[1:]  # 출력（정답 레이블）

# 모델 생성
model = SimpleRnnlm(vocab_size, wordvec_size, hidden_size)
optimizer = SGD(lr)
trainer = RnnlmTrainer(model, optimizer)

trainer.fit(xs, ts, max_epoch, batch_size, time_size)
trainer.plot()

# ./trainer.py